## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [120]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [121]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [122]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_280 (Conv2D)             (None, 32, 32, 16)   448         input_13[0][0]                   
__________________________________________________________________________________________________
batch_normalization_253 (BatchN (None, 32, 32, 16)   64          conv2d_280[0][0]                 
__________________________________________________________________________________________________
activation_400 (Activation)     (None, 32, 32, 16)   0           batch_normalization_253[0][0]    
___________________________________________________________________________________________

In [123]:
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

In [124]:
data_generator = ImageDataGenerator(
    featurewise_center=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False)

In [125]:
data_generator.fit(x_train)

In [126]:
model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size,
                    epochs=epochs)

Epoch 1/30
1563/1562 [==============================] - 144s 92ms/step - loss: 2.2655 - acc: 0.3929
Epoch 2/30
1563/1562 [==============================] - 127s 81ms/step - loss: 1.9290 - acc: 0.5032
Epoch 3/30
1563/1562 [==============================] - 125s 80ms/step - loss: 1.7765 - acc: 0.5510
Epoch 4/30
1563/1562 [==============================] - 125s 80ms/step - loss: 1.6681 - acc: 0.5834
Epoch 5/30
1563/1562 [==============================] - 132s 84ms/step - loss: 1.5767 - acc: 0.6153
Epoch 6/30
1563/1562 [==============================] - 132s 84ms/step - loss: 1.5056 - acc: 0.6350
Epoch 7/30
1563/1562 [==============================] - 128s 82ms/step - loss: 1.4469 - acc: 0.6533
Epoch 8/30
1563/1562 [==============================] - 125s 80ms/step - loss: 1.3948 - acc: 0.6720
Epoch 9/30
1563/1562 [==============================] - 125s 80ms/step - loss: 1.3471 - acc: 0.6840
Epoch 10/30
1563/1562 [==============================] - 127s 81ms/step - loss: 1.3055 - acc: 0.6992

In [127]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0551389245033265
Test accuracy: 0.7752


### 在Keras中使用圖像增強來進行深度學習

https://github.com/erhwenkuo/deep-learning-with-keras-notebooks/blob/master/1.0-image-augmentation.ipynb